# Multiple Inference Cookbook

This template is an 80-20 solution for multiple inference. It uses many of the latest techniques in multiple inference to answer the following questions:

1. *Compare to zero.* Which parameters are significantly different from zero?
2. *Compare to the average.* Which parameters are significantly different from the average (i.e., the average value across all parameters)?
3. *Pairwise comparisons.* Which parameters are significantly different from which other parameters?
4. *Ranking.* What is the ranking of each parameter?
5. *Best parameter identification.* Which parameters might be the largest (i.e., the highest ranked)?
6. *Inference after ranking.* What are the values of the parameters given their rank? e.g., What is the value of the parameter with the largest estimated value?
7. *Distribution.* What does the distribution of parameters look like?

Click the badge below to use this template on your own data. This will open the notebook in a Jupyter binder.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/dsbowen%2Fconditional-inference/HEAD?urlpath=lab/tree/docs/examples/multiple_inference.ipynb)

Instructions:

1. Upload a file named `data.csv` to this folder with your conventional estimates. Open `data.csv` to see an example. In this file, we named our dependent variable "dep_variable", and have estimated parameters named "policy0",..., "policy9". The first column of `data.csv` contains the conventional estimates $m$ of the unknown parameters. The remaining columns contain consistent estimates of the covariance matrix $\Sigma$. In `data.csv`, $m=(0, 1,..., 9)$ and $\Sigma = I$.
2. Modify the code if necessary.
3. Run the notebook.

### Runtime warnings and long running times

If you are estimating many parameters or the parameters effects are close together, you may see `RuntimeWarning` messages and experience long runtimes. Runtime warnings are common, usually benign, and can be safely ignored.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from multiple_inference.bayes import Improper, Nonparametric, Normal
from multiple_inference.confidence_set import ConfidenceSet, AverageComparison, PairwiseComparison, SimultaneousRanking
from multiple_inference.rank_condition import RankCondition

data_file = "data.csv"
alpha = .05

np.random.seed(0)
sns.set()

First, we'll summarize and plot your original (conventional) estimates.

In [ ]:
model = Improper.from_csv(data_file, sort=True)
results = model.fit(title="Conventional estimates")
results.summary(alpha=alpha)

In [ ]:
results.point_plot(alpha=alpha)
plt.show()

Next, we'll create a *reconstruction plot*. A reconstruction plot shows what we would expect the conventional estimates to look like if your estimates were correct.

For example, imagine we ran a randomized control trial in which we tested the effects of ten treatments. We then obtained our conventional estimates using OLS and rank ordered them. Now, suppose our conventional estimates are correct (i.e., assume the true effect of each treatment follows its OLS distribution). What would our new estimates look like if we reran the experiment, estimated the treatment effects by OLS, and rank ordered them again?

Ideally, the new conventional estimates would look like our original conventional estimates. Below, the blue dots show the simulated distribution of new estimates. The orange x's show our original conventional estimates. So, ideally, the blue dots should be on top of the orange x's.

Conventional estimates are more spread out than the true parameters in expectation. That is, conventional estimates often suggest that some parameters are very different from others, even if they are all similar. We call this problem *fictitious variation*. Our conventional estimates suffer from fictitious variation when the blue dots are more spread out than the orange x's.

In [ ]:
results.reconstruction_point_plot(title="Conventional estimates reconstruction plot")
plt.show()

## Compare to zero

First, we'll test which parameters significantly differ from zero. There are at least two criteria we could apply in answering this question. First, we might want to control the *false discovery rate*. The false discovery rate is the proportion of rejected hypotheses that are truly null. This testing procedure begins by converting p-values into *q-values*. We start by splitting the distribution of p-values into two components: a uniform distribution and a right-skewed distribution. (If a null hypothesis is true, its p-value will follow a uniform distribution. If a null hypothesis is false, its p-value will follow a right-skewed distribution.) Using these components, we can estimate the proportion of true nulls we would reject (the q-value) by rejecting all hypotheses with a p-value below a given significance threshold. Look at the "qvalue" column in the table below. Suppose we reject all null hypotheses with a q-value below 0.05. In that case, about 5\% of the rejected hypotheses are truly null.

Second, we might want to control the *family-wise error rate*. The family-wise error rate is the probability of making at least one false discovery. That is, it is the probability of rejecting at least true null hypothesis. We can control the family-wise error rate using *simultaneous confidence intervals*. Suppose we estimated $K$ parameters. Simultaneous confidence intervals begin by forming a $K$-dimensional rectangle such that all the parameters fall within this rectangle with 95\% probability. Then, they "project" this $K$-dimensional rectangle onto the dimension for a specific parameter to obtain that parameter's simultaneous confidence interval. By construction, all the parameters fall within their simultaneous confidence interval with 95\% probability. Look at the "pvalue" column in the table below. Suppose we reject all null hypotheses with a p-value below 0.05. In that case, there is a 5\% chance that any rejected hypotheses are truly null. Simultaneous confidence intervals are similar to a Bonferroni correction but have higher power. 

In [ ]:
model = ConfidenceSet.from_csv(data_file, sort=True)
results = model.fit()
results.summary(alpha=alpha)

In [ ]:
ax = results.point_plot(alpha=alpha)
ax.axvline(0, linestyle="--")
plt.show()

We can also use a stepdown improvement to increase power while controlling the family-wise error rate. This is analogous to Holm's stepdown improvement on the Bonferroni correction.

In [ ]:
results.test_hypotheses(alpha=alpha)

## Compare to the average

Next, we'll use q-values and simultaneous confidence intervals to discover which parameters significantly differ from the average (i.e., the average across all parameters). This is similar to what we did before. However, instead of constructing q-values and simultaneous confidence intervals for the parameters $\mu_k$, we'll construct them for the difference between the parameters and the average $\mu_k - \mathbf{E}[\mu_j]$.

In [ ]:
model = AverageComparison.from_csv(data_file, sort=True)
results = model.fit()
results.summary(alpha=alpha)

The x-axis in the plot below is the estimated difference between each parameter and the average $\mu_k - \mathbf{E}[\mu_j]$.

In [ ]:
ax = results.point_plot(alpha=alpha)
ax.axvline(0, linestyle="--")
plt.show()

In [ ]:
results.test_hypotheses(alpha=alpha)

## Pairwise comparisons

Next, we'll use q-values and simultaneous confidence intervals to perform pairwise comparisons. Instead of constructing q-values and simultaneous confidence intervals for the individual parameters $\mu_k$, we'll construct them for all pairwise differences $\mu_j - \mu_k \quad \forall j, k$.

First, we'll perform hypothesis testing using q-values by setting `criterion="fdr"` for "false discovery rate." Green squares in the heatmap below indicate that we have rejected the null hypothesis $H_{j,k}: \mu_k = \mu_j$, where $j$ is the parameter on the x-axis and $k$ is the parameter on the y-axis. Red squares indicate that we have not rejected the null hypothesis. Of the rejected hypotheses, we expect that at most $\alpha$ proportion are truly null.

In [ ]:
model = PairwiseComparison.from_csv(data_file, sort=True)
results = model.fit()
results.hypothesis_heatmap(alpha=alpha, triangular=True, criterion="fdr")
plt.show()

Second, we'll perform hypothesis testing using simultaneous confidence intervals by setting `criterion="fwer"` for "family-wise error rate." Green squares in the heatmap below indicate that we have rejected the null hypothesis $H_{j,k}: \mu_j \leq \mu_k$. That is, we have concluded that the parameter on the x-axis is greater than the parameter on the y-axis. Red squares indicate that we have not rejected the null hypothesis. We expect that we have rejected at least one truly will hypothesis with probability $\alpha$.

In [ ]:
results.hypothesis_heatmap(alpha=alpha, triangular=True, criterion="fwer")
plt.show()

## Ranking

Next, we'll estimate each parameter's rank. For example, we might say that we're 95% confident that the parameter with the largest estimated value is genuinely one of the largest three parameters. The simplest way to estimate ranks by performing pairwise comparisons with simultaneous confidence intervals. For example, if there's a 95% chance that $\mu_k$ was worse than one parameter and better than $K-5$ parameters, then there's a 95% chance that parameter $\mu_k$ ranks between second and fifth.

The results below come from a similar but more efficient procedure, so they might not precisely match the pairwise comparisons plot above.

In [ ]:
model = SimultaneousRanking.from_csv(data_file, sort=True)
results = model.fit()
results.summary(alpha=alpha)

In [ ]:
results.point_plot(alpha=alpha)
plt.show()

## Best parameter identification

Next, we'll compute a set of parameters containing the truly largest parameter with 95% confidence. The simplest way to calculate this set is to use pairwise comparisons. If we cannot reject the hypothesis that $\mu_j$ is greater than $\mu_k$ for any $j$, then $\mu_k$ is in this set. That is, we cannot reject the hypothesis that $\mu_k$ is the largest parameter with 95% confidence.

Again, the results below come from a similar but more efficient procedure, so they might not precisely match the pairwise comparisons plot above.

Parameters with "True" next to them in the table below are in this set. Parameters with "False" next to them are not.

In [ ]:
results.compute_best_params(alpha=alpha)

## Inference after ranking

Next, we'll estimate parameters given the rank of their conventional estimates. For example, imagine we ran a randomized control trial testing ten treatments and want to estimate the effectiveness of the top-performing treatment.

One property we want our estimators to have is *quantile-unbiasedness*. An estimator is quantile-unbiased if the true parameter falls below its $\alpha$-quantile estimate with probability $\alpha$ given its estimated rank. For example, the true effect of the top-performing treatment should fall below its median estimate half the time.

Similarly, we want confidence intervals to have *correct conditional coverage*. Correct conditional coverage means that the parameter should fall within our $\alpha$-level confidence interval with probability $1-\alpha$ given its estimated rank. For example, the true effect of the top-performing treatment should fall within its 95% confidence interval 95% of the time.

Below, we compute the optimal quantile-unbiased estimates and conditionally correct confidence intervals for each parameter given its rank.

In [ ]:
model = RankCondition.from_csv(data_file, sort=True)
results = model.fit(title="Conditional estimates")
results.summary(alpha=alpha)

In [ ]:
results.point_plot(alpha=alpha)
plt.show()

Conditional inference is a strict requirement. Conditionally quantile-unbiased estimates can be highly variable. And conditionally correct confidence intervals can be unrealistically long. We can often obtain more reasonable estimates by focusing on *unconditional* inference instead of *conditional* inference.

Imagine we ran our randomized control trial 10,000 times and want to estimate the effect of the top-performing treatment. We need *conditional* inference if we're interested the subset of trials where a specific parameter $k$ was the top performer. However, we can use *unconditional* inference if we're only interested in being right "on average" across all 10,000 trials.

Below, we use *hybrid estimates* to compute approximately quantile-unbiased estimates and unconditionally correct confidence intervals for each parameter.

If you don't know whether you need conditional or unconditional inference, use unconditional inference.

In [ ]:
results = model.fit(beta=.005, title="Hybrid estimates")
results.summary(alpha=alpha)

In [ ]:
results.point_plot(alpha=alpha)
plt.show()

## Distributions

Next, we'll use Bayesian estimators to understand the distribution of parameters. Bayesian estimators begin with a *prior* distribution and then update that belief based on data using Bayes' Theorem to obtain a *posterior* distribution.

Classical Bayesian estimators take the prior as given. However, we can often obtain better estimates by using empirical Bayes to estimate the prior from the data. For example, imagine predicting MLB players' on-base percentage (OBP) next season. We might predict that a player's OBP next season will be the same as his OBP in the previous season. But how can we predict the OBP for a rookie with no batting history? One solution is to predict that the rookie's OBP will be similar to last season's rookies' OBP. In Bayesian terms, we've constructed a prior belief about *next* season's rookies' OBP using data from the *previous* season's rookies' rookies' OBP.

Empirical Bayes uses the same logic. Imagine randomly selecting one parameter $\mu_k$ and putting the data we used to estimate it in a locked box. We can use the data about the remaining parameters to estimate a prior distribution for $\mu_k$.

Empirical Bayes estimators can be parametric or non-parametric. Parametric empirical Bayes assumes the shape of the prior distribution. Nonparametric empirical Bayes does not assume the shape of the prior distribution. Below, we apply a parametric empirical Bayes estimator assuming a normal prior distribution.

First, let's plot one parameter's prior, conventional, and posterior distributions.

In [ ]:
model = Normal.from_csv(data_file, sort=True)
results = model.fit()
results.line_plot(0)
plt.show()

Next, let's create a reconstruction plot for the Bayesian estimator. This plot shows what we would expect the conventional estimates to look like if the Bayesian model were correct. Remember that we want the blue dots to be on top of the orange x's.

Compare the reconstruction plot for the Bayesian estimates (below) to the reconstruction plot for the conventional estimates near the top of the notebook. Looking at the reconstruction plot for the conventional estimates at the top of the notebook, you'll likely see that the conventional estimates are too spread out (the blue dots are more spread out than the orange x's). Looking at the reconstruction plot for the Bayesian estimates below, you'll likely see that the Bayesian estimates are appropriately spread out (the blue dots are on top of the orange x's).

In [ ]:
results.reconstruction_point_plot(title="Parametric empirical Bayes reconstruction plot")
plt.show()

Now, let's summarize and plot the Bayesian estimates.

Both Bayesian and rank condition estimators give parameter estimates and confidence intervals. Which ones should we use?

We should use Bayesian estimators if our goal is to understand the distribution of parameters. If, however, our goal is to estimate a parameter given its rank (e.g., the top-performing parameter), the answer is less clear. In my experience, Bayesian point estimates are better than rank condition point estimates, but rank condition confidence intervals are better than Bayesian confidence intervals. For example, if you want to estimate the top-performing parameter, I recommend writing your discussion section using the Bayesian point estimate and the rank condition (hybrid) confidence interval.

This template is an 80-20 solution for multiple inference. For a 90-50 solution, use cross-validation to decide whether Bayesian or rank condition estimates are better for your data set.

What if our parametric assumption is incorrect? That is, what if the prior distribution isn't normal?

Amazingly, parametric empirical Bayes estimators usually give us more accurate point estimates than conventional estimators regardless of the prior distribution. Some, like the James-Stein estimator, are mathematically guaranteed to do so. Additionally, we can estimate "robust" confidence intervals that have correct coverage regardless of the prior distribution.

In [ ]:
results.summary(alpha=alpha, robust=True)

In [ ]:
results.point_plot(alpha=alpha, robust=True)
plt.show()

We can also use Bayesian estimators to estimate the rank of each parameter. Bayesian estimators are usually more confident about the parameter ranks than our previous ranking analysis suggests. Our previous ranking analysis suggested substantial uncertainty about parameter rankings because it strictly controlled false positives. Bayesian estimates of parameter rankings do not control for false positives but give tighter and often more reasonable estimates. In my experience, the previous ranking analysis is more appropriate for hypothesis testing, while Bayesian ranking analysis is more appropriate in terms of objectives like Brier scores and log loss.

In [ ]:
results.rank_df

Finally, we'll repeat this exercise with a nonparametric empirical Bayes estimator. Remember, parametric empirical Bayes estimators assume the shape of the prior distribution (e.g., normal). Nonparametric empirical Bayes estimators do not assume the shape of the prior distribution.

Should you use a parametric or nonparametric empirical Bayes estimator? In my experience, parametric empirical Bayes is better for a small number of parameters, and nonparametric empirical Bayes is better for a large number of parameters. If both estimators give similar point estimates, trust the one with wider confidence intervals (usually the parametric version). If the estimators give very different point estimates, my rule of thumb is to trust parametric empirical Bayes when estimating fewer than 50 parameters and nonparametric empirical Bayes otherwise.

But again, for a 90-50 solution, use cross-validation to decide between parametric and nonparametric empirical Bayes.

In [ ]:
model = Nonparametric.from_csv(data_file, sort=True)
results = model.fit()
results.line_plot(0)
plt.show()

In [ ]:
results.reconstruction_point_plot(title="Nonparametric empirical Bayes reconstruction plot")
plt.show()

In [ ]:
results.summary(alpha=alpha)

In [ ]:
results.point_plot(alpha=alpha)
plt.show()

In [ ]:
results.rank_df